<a href="https://colab.research.google.com/github/anirudh201098/Object-detection-with-Yolo-models/blob/main/Yolo%20models/YOLO_darknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://cdn.analyticsvidhya.com/wp-content/uploads/2018/12/yologo_2.png)

#Pre-requisites :
A) YOLO requires image annotations in .txt extension in the following format: 


```
<object-class> <x> <y> <height> <width>
For example if tiger is object class 0 in an image "img1.png" 
then annotation file will also have the name img1.txt 
and annotation in the file will be this for one bounding box:

0 0.295491 0.631188 0.589920 0.502829


```


B) All the images and respective annotation file should be present in the same directory. 



**PSACAL VOC 2012 dataset: http://host.robots.ox.ac.uk/pascal/VOC/voc2012/index.html**



Introduction

The main goal of this challenge is to recognize objects from a number of visual object classes in realistic scenes (i.e. not pre-segmented objects). It is fundamentally a supervised learning learning problem in that a training set of labelled images is provided. The twenty object classes that have been selected are:

    Person: person
    Animal: bird, cat, cow, dog, horse, sheep
    Vehicle: aeroplane, bicycle, boat, bus, car, motorbike, train
    Indoor: bottle, chair, dining table, potted plant, sofa, tv/monitor


**1. Mount the drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**2. List the files and directories at current path**

In [ ]:
%ls  # List all the fies and directories 

drive/  sample_data/


**3. Move to the folder where dataset and main files are present**

In [ ]:
%cd /content/drive/My\ Drive/Research_Internship

/content/drive/My Drive/Research_Internship


**4. List the files and directories at current path**

In [ ]:
%ls 

darknet/  Yolo-2/  Yolo-3/


**5. Clone darknet** (uncomment when running it for first time after clone comment it again)

In [ ]:
# !git clone https://github.com/AlexeyAB/darknet/     # cloning darknet from github

In [ ]:
%cd darknet  

/content/drive/My Drive/Research_Internship/darknet


**6. List the files and directories at current path**

In [ ]:
!ls

3rdparty	       DarknetConfig.cmake.in	predictions.jpg
backup		       darknet.py		README.md
bad.list	       darknet_video.py		result.avi
build		       data			results
build.ps1	       dataset			scripts
build.sh	       extraction.conv.weights	sed2jULfN
cfg		       image_yolov2.sh		sed2tyMGH
chart.png	       image_yolov3.sh		sedveAFiM
chart_voc-custom.png   include			sedziVXLS
cmake		       json_mjpeg_streams.sh	src
CMakeLists.txt	       LICENSE			video_v2.sh
custom-yolo2.backup    Makefile			video_yolov3.sh
darknet		       net_cam_v3.sh		yolov4.conv.137
darknet19_448.conv.23  obj


**7. Activate GPU, CUDNN and OPENCV in makefile** (Uncomment it for first time run then again comment it)

In [ ]:
 !sed -i 's/GPU=0/GPU=1/g' Makefile
 !sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
 !sed -i 's/OPENCV=0/OPENCV=1/g' Makefile

**8. Build executable code using make** (Uncomment it for first time run after that again comment below make command)

In [ ]:
 !make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1130:10: note: ...thi

**9. Change the permission for dakrnet file**

In [ ]:
!chmod +x ./darknet  #change made of darknet file for execution (+x -> executable)

**10. Setting up the configuration**

**(A) Altering .cfg file**

It contains the model architecture information.

**Download yolo.train.cfg file from cfg folder of yolov1**



**Open the file using any editor and do following changes:**
*   change line batch to batch=64
*   change line subdivisions to subdivisions=4
*   height =448, width=448 or any value multiple of 32
*   change line max_batches to 40000(classes*2000 i.e 20x2000)
*   change line steps steps=200,400,600,20000,30000
*   change line classes=20 to your number of objects in [detection] layer
 

**(B) Creating .names file**

*   It contains the class information
*   Create a new file in your editor named it as voc.names
*   In the file write the name of classes(each class name in new line)

```
voc.names:

aeroplane
bicycle
bird
boat
bottle
bus
car
cat
chair
cow
diningtable
dog
horse
motorbike
person
pottedplant
sheep
sofa
train
tvmonitor
```

**(C) Creating .data file**

It contains information about path of training file, testing file, class name file and path where to store the trained weights 

```
voc.data
classes= 20
train  = data/voc/train.txt    => Train file path
valid  = data/test.txt  => Test file path
names = data/voc.names => Classes file path
backup = backup/   => Weights will be saved after every 1000 epochs in this directory.
```
Make sure you save the train and test .txt  files in data folder of darknet


**(D) Downloading the Imagenet weights for YOLO V1**

In [ ]:
!wget http://pjreddie.com/media/files/extraction.conv.weights

--2020-07-27 18:57:08--  http://pjreddie.com/media/files/extraction.conv.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pjreddie.com/media/files/extraction.conv.weights [following]
--2020-07-27 18:57:09--  https://pjreddie.com/media/files/extraction.conv.weights
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89721616 (86M) [application/octet-stream]
Saving to: ‘extraction.conv.weights’

extraction.conv.wei 100%[===================>]  85.56M   215KB/s    in 5m 30s  

2020-07-27 19:02:39 (266 KB/s) - ‘extraction.conv.weights’ saved [89721616/89721616]



**11. Before you proeed ahead upload the modifiled files:**

*   Upload voc.names and voc.data inside data folder of darknet
*   Upload voc.cfg inside cfg folder of darknet

**12. Install dos2unix library to change files as per unix compatabiity**

In [ ]:
!sudo apt-get install dos2unix 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 11 not upgraded.
Need to get 351 kB of archives.
After this operation, 1,267 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dos2unix amd64 7.3.4-3 [351 kB]
Fetched 351 kB in 1s (369 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package dos2unix.
(Reading database ... 144628 files and directories curren

**13. Convert the uploaded file in unix format**

If you get an error here, then recheck you paths for all the files and paste them accordingly.

In [ ]:
 !dos2unix /content/drive/My\ Drive/Research_Internship/darknet/data/voc/train.txt
 !dos2unix /content/drive/My\ Drive/Research_Internship/darknet/data/test.txt
 !dos2unix /content/drive/My\ Drive/Research_Internship/darknet/data/voc.data
 !dos2unix /content/drive/My\ Drive/Research_Internship/darknet/data/voc.names
 !dos2unix /content/drive/My\ Drive/Research_Internship/darknet/cfg/yolov1/yolo.train.cfg

dos2unix: converting file /content/drive/My Drive/Research_Internship/darknet/data/voc/train.txt to Unix format...
dos2unix: converting file /content/drive/My Drive/Research_Internship/darknet/data/test.txt to Unix format...
dos2unix: converting file /content/drive/My Drive/Research_Internship/darknet/data/voc.data to Unix format...
dos2unix: converting file /content/drive/My Drive/Research_Internship/darknet/data/voc.names to Unix format...
dos2unix: converting file /content/drive/My Drive/Research_Internship/darknet/cfg/yolov1/yolo.train.cfg to Unix format...


**14. Training dataset** (When training the model from pre-trained weights use below command )

Since I have already trained the PASCAL VOC 2012 train data for 38000 epochs, I am using the following command.

If you are training the data for the first time, then change the code to 
```
!./darknet yolo train cfg/yolov1/yolo.train.cfg extraction.conv.weights  -dont_show

```

If you have dataset which has similar classes to PASCAL VOC 2012 dataset, you can use pre-trained weights from the following link: **https://drive.google.com/file/d/10gGE2AMKtTjIj40zyEqlkRYQPaQ8gu7U/view?usp=sharing**

In [ ]:
!./darknet yolo train cfg/yolov1/yolo.train.cfg backup/yolo_38000.weights  -dont_show

Streaming output truncated to the last 5000 lines.
Detection Avg IOU: 0.833992, Pos Cat: 0.913357, All Cat: 0.049220, Pos Obj: 0.494285, Any Obj: 0.010954, count: 23
Detection Avg IOU: 0.690217, Pos Cat: 0.928224, All Cat: 0.049404, Pos Obj: 0.386861, Any Obj: 0.016087, count: 45
Detection Avg IOU: 0.660450, Pos Cat: 0.868516, All Cat: 0.049359, Pos Obj: 0.356494, Any Obj: 0.014308, count: 35
Detection Avg IOU: 0.730221, Pos Cat: 0.872477, All Cat: 0.049718, Pos Obj: 0.472787, Any Obj: 0.013436, count: 37
39168: 2.471243, 2.431997 avg, 0.000050 rate, 1.860683 seconds, 2506752 images
Loaded: 0.479048 seconds
Detection Avg IOU: 0.760927, Pos Cat: 0.901632, All Cat: 0.050379, Pos Obj: 0.479189, Any Obj: 0.014504, count: 32
Detection Avg IOU: 0.853794, Pos Cat: 0.970308, All Cat: 0.050351, Pos Obj: 0.608769, Any Obj: 0.011670, count: 19
Detection Avg IOU: 0.718986, Pos Cat: 0.857772, All Cat: 0.049953, Pos Obj: 0.504109, Any Obj: 0.013688, count: 27
Detection Avg IOU: 0.721724, Pos Cat: 0.

**15. Calculate the Mean average Precision of the trained model**

In [ ]:
 !./darknet detector map data/voc.data cfg/yolov1/yolo.train.cfg backup/yolo_40000.weights  -dont_show -points 0 

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     64       7 x 7/ 2    448 x 448 x   3 ->  224 x 224 x  64 0.944 BF
   1 max                2x 2/ 2    224 x 224 x  64 ->  112 x 112 x  64 0.003 BF
   2 conv    192       3 x 3/ 1    112 x 112 x  64 ->  112 x 112 x 192 2.775 BF
   3 max                2x 2/ 2    112 x 112 x 192 ->   56 x  56 x 192 0.002 BF
   4 conv    128       1 x 1/ 1     56 x  56 x 192 ->   56 x  56 x 128 0.154 BF
   5 conv    256       3 x 3/ 1     56 x  56 x 128 ->   56 x  56 x 256 1.850 BF
   6 conv    256       1 x 1/ 1     56 x  56 x 256 ->   56 x  56 x 256 0.411 BF
   7 conv    512       3 x 3/ 1     56 x  56 x 256 ->   56 x  56 x 512 7.399 BF
   8 max                2x 2/ 2     56 x  56 x 512 ->   

**16. Predicting bounding boxes for new image:**

The predicted image will be saved in ./darknet folder with the name predictions.jpg

In [ ]:
!./darknet yolo test cfg/yolov1/yolo.train.cfg backup/yolo_40000.weights data/dog.jpg  -dont_show 

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 16, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     64       7 x 7/ 2    448 x 448 x   3 ->  224 x 224 x  64 0.944 BF
   1 max                2x 2/ 2    224 x 224 x  64 ->  112 x 112 x  64 0.003 BF
   2 conv    192       3 x 3/ 1    112 x 112 x  64 ->  112 x 112 x 192 2.775 BF
   3 max                2x 2/ 2    112 x 112 x 192 ->   56 x  56 x 192 0.002 BF
   4 conv    128       1 x 1/ 1     56 x  56 x 192 ->   56 x  56 x 128 0.154 BF
   5 conv    256       3 x 3/ 1     56 x  56 x 128 ->   56 x  56 x 256 1.850 BF
   6 conv    256       1 x 1/ 1     56 x  56 x 256 ->   56 x  56 x 256 0.411 BF
   7 conv    512       3 x 3/ 1     56 x  56 x 256 ->   56 x  56 x 512 7.399 BF
   8 max                2x 2/ 2     56 x  56 x 512 -> 

In [ ]:
!./darknet yolo test cfg/yolov1/yolo.train.cfg backup/yolo_final.weights data/dog.jpg  -dont_show 

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 16, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     64       7 x 7/ 2    448 x 448 x   3 ->  224 x 224 x  64 0.944 BF
   1 max                2x 2/ 2    224 x 224 x  64 ->  112 x 112 x  64 0.003 BF
   2 conv    192       3 x 3/ 1    112 x 112 x  64 ->  112 x 112 x 192 2.775 BF
   3 max                2x 2/ 2    112 x 112 x 192 ->   56 x  56 x 192 0.002 BF
   4 conv    128       1 x 1/ 1     56 x  56 x 192 ->   56 x  56 x 128 0.154 BF
   5 conv    256       3 x 3/ 1     56 x  56 x 128 ->   56 x  56 x 256 1.850 BF
   6 conv    256       1 x 1/ 1     56 x  56 x 256 ->   56 x  56 x 256 0.411 BF
   7 conv    512       3 x 3/ 1     56 x  56 x 256 ->   56 x  56 x 512 7.399 BF
   8 max                2x 2/ 2     56 x  56 x 512 -> 

**17. Prediction for videos:**

In [ ]:
!./darknet detector demo data/voc.data cfg/yolov1/yolo.train.cfg  backup/yolo_40000.weights data/traffic.mp4 -out_filename yolo1_pred.avi -dont_show

Streaming output truncated to the last 5000 lines.
 cvWriteFrame 
Objects:

car: 28% 

FPS:76.9 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

person: 41% 
car: 33% 

FPS:76.6 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

person: 39% 
car: 31% 

FPS:76.7 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

car: 37% 

FPS:76.1 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

car: 35% 

FPS:76.2 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

car: 39% 

FPS:76.1 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

car: 40% 

FPS:75.8 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

car: 42% 

FPS:76.0 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

car: 36% 

FPS:76.0 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

car: 33% 

FPS:75.3 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

car: 36% 

FPS:75.2 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

person: 30% 
car: 41% 

FPS:75.0 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

person: 27% 
car: 39% 

FPS:74.9 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

person: 27% 
car: 33% 

FPS:75.1 	 AVG_FPS:76.2

 cvWriteFrame 
Objects:

perso